In [ ]:
import pandas as pd
import io
import requests
import datetime
import plotly.express as px

## Get the Timeseries Data

Add documentation about the time series here.

In [ ]:
from functions import get_time_series

confirmed, deaths, recovered, time_series_dates = get_time_series(local=True)

## Get all the Daily Reports 

Add documentation about these reports here.

In [ ]:
from functions import get_daily_reports

daily_report_data, daily_report_dates = get_daily_reports(local=True)

## All Done Gathering Data.

Let's merge the time_series_data into one dataframe

In [ ]:
from model import data_by_area
from data import us_confirmed 

data = data_by_area(area='Nebraska', col='State', df=us_confirmed)
data.plot()

In [ ]:

def data_by_area(area='US', col='Country/Region', df=None):
	from data import time_series_date_list
	print(col)
	data =pd.Series(
		[df.loc[(df[col] == area)][date].sum() for date in time_series_date_list],
		)
	return data


In [ ]:
data_by_area(area='US', df=confirmed).plot()

In [ ]:
from functions import make_country_labels, make_state_labels

from data import confirmed

data = confirmed.copy()
print(data.groupby('Country/Region').sum().iloc[:,-1].sort_values(ascending=False).index)


make_country_labels(data=confirmed)

In [ ]:
data.groupby('Country/Region').sum().iloc[:,-2].sort_values(ascending=False)

In [ ]:
import pandas as pd
def get_daily_reports(local=True):
    start_date = pd.to_datetime('01/22/2020')
    end_date = pd.to_datetime('today')
    dates = pd.date_range(start_date, end_date)
    daily_reports = {}
    all_reports = []
    if local == True:
        print('Getting daily reports using local data')
        for date in dates:
            date_str = date.strftime('%m-%d-%Y')
            file_name = 'data/' + date_str + '.csv'
            try:
                df = pd.read_csv(file_name, header=0)
                df['Date'] = date_str
                all_reports.append(df)
            except:
                print("Failed to load {file}".format(file=file_name))
    elif local == False:
        print('Getting daily reports using on-line data')
        daily_report_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'
        for date in dates:
            # File format: 'MM-DD-YYYY.csv'
            date_str = date.strftime('%m-%d-%Y')
            file_name =date_str + '.csv'
            url = daily_report_url + file_name
            try:
                df = pd.read_csv(file_name, header=0)
                df['Date'] = date_str
                all_reports.append(df)
            except:
                print("Failed to load {file}".format(file=file_name))
    daily_reports = pd.concat(all_reports, axis=0, ignore_index=True)
#    valid_dates = [pd.to_datetime(date) for date in list(daily_reports.keys())]
    return daily_reports

In [ ]:
df = get_daily_reports()

df['Date']
df.Date = pd.to_datetime(df.Date)
#df['Last Update'] = pd.to_datetime(df['Last Update'])
#df['Last Update']

In [ ]:
print(df.columns)
area = 'US'
most_recent = df[df['Country/Region'] == area]['Last Update'].max() - pd.Timedelta('12 hours')
df[(df['Country/Region'] == area) & (df['Last Update'] >= most_recent)].sort_values('Last Update', ascending=False)

In [ ]:
def recent_updates(area='US', timedelta='12 hours', sort='Confirmed', df=None):
    ascending = {
        'Province/State': True
    }
    most_recent = df[df['Country/Region'] == area]['Last Update'].max() - pd.Timedelta(timedelta)
    return df[
        (df['Country/Region'] == area) & (df['Last Update'] >= most_recent)].sort_values(
            sort, ascending=ascending.get(sort, False))


In [6]:
from model import data_by_area
from data import time_series_dates, confirmed, deaths, get_states
import numpy as np

df_c = confirmed.groupby('Country/Region').sum()
df_d = deaths.groupby('Country/Region').sum()

In [20]:
import pandas as pd
def growth_rate(
        df=None,
        ndays=7,
        area=None,
        col='Country/Region',
        do_sort=True):
    data = df[df.columns[-ndays:-1]].copy()
    data = data.sort_values(list(data.columns)[-1:], ascending=False)
    rates = (data.diff(axis=1)/data)
    rates = rates[rates.columns[1:]]
    if do_sort:
        rates = rates.sort_values(list(rates.columns)[-1:], ascending=False)
    if area:
        rates = rates[col][area]
    return rates


In [21]:
rates = growth_rate(df=df_c)
print(rates)

                                3/10/20  3/11/20  3/12/20  3/13/20  3/14/20
Country/Region                                                             
Rwanda                              NaN      NaN      NaN      NaN      1.0
Eswatini                            NaN      NaN      NaN      NaN      1.0
Uruguay                             NaN      NaN      NaN      NaN      1.0
Ghana                               NaN      NaN      NaN      NaN      1.0
Seychelles                          NaN      NaN      NaN      NaN      1.0
...                                 ...      ...      ...      ...      ...
Central African Republic            NaN      NaN      NaN      NaN      NaN
Kosovo                              NaN      NaN      NaN      NaN      NaN
Congo (Brazzaville)                 NaN      NaN      NaN      NaN      NaN
Equatorial Guinea                   NaN      NaN      NaN      NaN      NaN
occupied Palestinian territory      NaN      NaN      NaN      NaN      NaN

[147 rows x

In [1]:
pandemic = 100
pandemic_confirmed = df_c[df_c.iloc[:,-1]>pandemic]
pandemic_deaths = df_d[df_c.iloc[:,-1]>pandemic]

case_rates = growth_rate(df=pandemic_confirmed)
death_rates = growth_rate(df=pandemic_deaths)



NameError: name 'df_c' is not defined

In [28]:
import plotly.graph_objects as go
fig = go.Figure()
fig.layout.title = 'tets'

In [47]:
from data import time_series_date_list
ndays = 10
print(time_series_date_list[-1])
{str(date): str(date) for date in time_series_date_list[-ndays:]}

import numpy as np
[a for a in np.arange(0,2200,200)][1:]

3/15/20


[200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]

In [52]:
pandemic = 100
pandemic_countries = all_countries[all_countries['3/13/20'] >= pandemic].copy()

#confirmed[confirmed.columns[4:]].diff(axis=1)/confirmed[confirmed.columns[4:]].plot()

ndays = 7

data = pandemic_countries[pandemic_countries.columns[-ndays:-1]].copy()
data = data.sort_values(list(data.columns)[-1:], ascending=False)

rates = (data.diff(axis=1)/data)
rates = rates[rates.columns[1:]]
rates = rates.sort_values(list(rates.columns)[-1:], ascending=False)



from math import log
doubling_times = log(2)/rates
doubling_times[doubling_times == np.inf]=None
#print(doubling_times.mean(axis=1).sort_values())


In [14]:
from data import data_df, dates
import pandas as pd
import numpy as np

variables = dict(
    confirmed='Total Confirmed Cases',
    deaths='Total Deaths',
    recovered='Total Recovered Cases',
    new_confirmed='New Confirmed Cases',
    new_deaths='New Deaths',
    new_recovered='New Recovered Cases',
    case_rate='% Increase in Confirmed Cases',
    death_rate='% Increase in Deaths',
    case_mortality='Cumulative Case Mortality Rate',
    case_recovery='Cumulative Case Recovery Rate',
    case_doubling='Doubling Time for Confirmed Cases',
    death_doubling='Doubling Time of Deaths'
)

date_value = '2/22/20'
dff = data_df[data_df['date'] == date_value].copy()

xaxis_column_name = 'confirmed'
dff[dff['variable'] == xaxis_column_name]['value']

date_strings = [date.strftime('%m/%d/%y') for date in dates]
print(date_strings)

{i: date_strings[i] for i in list(np.arange(0,len(date_strings),7)) + [len(date_strings)-1]}

['01/22/20', '01/23/20', '01/24/20', '01/25/20', '01/26/20', '01/27/20', '01/28/20', '01/29/20', '01/30/20', '01/31/20', '02/01/20', '02/02/20', '02/03/20', '02/04/20', '02/05/20', '02/06/20', '02/07/20', '02/08/20', '02/09/20', '02/10/20', '02/11/20', '02/12/20', '02/13/20', '02/14/20', '02/15/20', '02/16/20', '02/17/20', '02/18/20', '02/19/20', '02/20/20', '02/21/20', '02/22/20', '02/23/20', '02/24/20', '02/25/20', '02/26/20', '02/27/20', '02/28/20', '02/29/20', '03/01/20', '03/02/20', '03/03/20', '03/04/20', '03/05/20', '03/06/20', '03/07/20', '03/08/20', '03/09/20', '03/10/20', '03/11/20', '03/12/20', '03/13/20', '03/14/20', '03/15/20']


{0: '01/22/20',
 7: '01/29/20',
 14: '02/05/20',
 21: '02/12/20',
 28: '02/19/20',
 35: '02/26/20',
 42: '03/04/20',
 49: '03/11/20',
 53: '03/15/20'}

In [19]:
from data import dates
import pandas as pd
import datetime

t = datetime.datetime.now()
int(t.strftime('%Y%m%d'))

#{str(date.strftime('%m/%d/%y')): str(date.strftime('%m/%d/%y')) for date in dates}

20200316